In [12]:
import pandas as pd
import numpy as np
import pickle

from crossref.restful import Works, Etiquette
from scihub_blastoise import SciHub_HydroPump


In [2]:
with open("required_5k_df", "rb") as fp:
    required_5k_df = pickle.load(fp)

with open("all_dois_5k", "rb") as fp:
    all_dois_5k = pickle.load(fp)

In [ ]:
with open("new_req_cr_results", "rb") as fp:
    new_req_cr_results = pickle.load(fp)

with open("new_req_sh_results", "rb") as fp:
    new_req_sh_results = pickle.load(fp)

In [ ]:

sh_h = SciHub_HydroPump()

mail = 'sduzenli@yandex.com.tr'

my_etiquette = Etiquette('Analysing Publishing Delay in Academic Journals', 'v2.0', 'https://github.com/Spidey0023/THEsis-Codes', mail)
ogi_works = Works(etiquette=my_etiquette)



partial_df = required_5k_df.loc["1_26384":,["ISSN","Year", "Required_Count_Sum","CR_Count"]]

partial_df.loc[:,"ISSN"] = partial_df.ISSN.map(list)
pr_dict = partial_df.to_dict("index")

# new_req_cr_results = list()
# new_req_sh_results = list() 


useful_cols = ['DOI', 'references-count', 'publisher', 'published-print', 'is-referenced-by-count', 'title', 'author', 'published-online', 'container-title', 'issued', 'ISSN', 'subject']

for key in pr_dict:
    issn = pr_dict[key]["ISSN"]
    year = pr_dict[key]["Year"]

    rc = pr_dict[key]["Required_Count_Sum"]
    crc = pr_dict[key]["CR_Count"]
    min_rc = min(rc+3, crc)

    loop_meta = [w for w in ogi_works.filter(issn=issn, from_pub_date=year, until_pub_date=year, type="journal-article").sample(int(min_rc)).select(useful_cols)]
    loop_meta = [dict(item, JRNL_ID=key) for item in loop_meta if item["DOI"] not in all_dois_5k]
    new_req_cr_results.extend(loop_meta)

    for artcl in loop_meta:
        doi = artcl["DOI"]
        pdf_url, date, kw = sh_h.get_dates_kws(doi)
        new_req_sh_results.append(dict(DOI = doi, direct_url = pdf_url, Dates = date, Keywords = kw))


In [ ]:
with open("new_req_cr_results_final","wb") as fp:
   pickle.dump(new_req_cr_results, fp)

with open("new_req_sh_results_final","wb") as fp:
   pickle.dump(new_req_sh_results, fp)